In [1]:
import requests
import os
import pandas as pd
from datetime import date, datetime, timedelta
import time
from io import StringIO
idag = date.today().isoformat()[0:10]
tokenCanvas = os.environ['tokenCanvas']

headers = {'Authorization': f'Bearer {tokenCanvas}',}
params = {'per_page': 100,}

# Initialisering

Først må eg lese fila over alle som er aktive lærarar i Canvas, og sjekke om dei er medlem i eit testemne. Dette vert berre gjort ein gang, og resultatet vert lagra i ei fil. 


In [ ]:
data = pd.read_csv("users_filtered.csv")
data['brukarnamn'] = data['login_id'].apply(lambda x: x.split('@')[0])
eksisterande = data[data['brukarnamn'].str.isalpha()]
eksisterande.info()

Den neste kodesnutten lager ei liste over alle lærarar, og kva testemne dei eventuelt er med i.

Køyrt og rensa 250526 - det skal ikkje vere nødvendig å køyre den fleire ganger. Resultatet etter vasking er lagra i `250526 oversikt.csv`.

In [3]:
# for person in eksisterande['login_id']:
#     print(f"{person} | ", end="")
#     try:
#         params = {'per_page': 100}
#         url = f"https://hvl.instructure.com/api/v1/accounts/1/users?search_term={person}"
#         respons = requests.get(url, headers=headers, params=params)
#         user_id = respons.json()[0]['id']
#         name = respons.json()[0]['name']
#         print(f"{user_id} | {name} | ", end="")
#         emneliste = []
#         params = {'per_page': 100, 'state': ['unpublished']}
#         url = f"https://hvl.instructure.com/api/v1/users/{user_id}/courses"
#         respons = requests.get(url, headers=headers, params=params)
#         for emne in respons.json():
#             emneliste.append(emne['name'])
#         params = {'per_page': 100, 'state': ['available']}
#         url = f"https://hvl.instructure.com/api/v1/users/{user_id}/courses"
#         respons = requests.get(url, headers=headers, params=params)
#         for emne in respons.json():
#             emneliste.append(emne['name'])
#         for e in emneliste: 
#             if 'Testemne' in e:
#                 print(f"{e}, ", end=" ")
#         print("")
#     except:
#         print("Ikkje lærar.") 

## Opprette testemne for alle lærarar som ikkje har det.

Eg må gå gjennom fila frå forrige blokk og plukke ut alle som har "Lag nytt emne" i Kontrollfelt:


In [10]:
data = pd.read_csv("250526 oversikt.csv")
data.head()

,brukar,id,namn,testemne
0,jwin@hvl.no,66156,Julia Noelle Winslow,Testemne for Julia Noelle Winslow
1,mabe@hvl.no,4576,Magnar Berge,Testemne for Magnar Berge Testemne for eigen...
2,gobe@hvl.no,1611,Geir Omar Berland,Testemne for Geir Omar Berland Testemne for ...
3,bvd@hvl.no,4401,Britt Viola Danielsen,Testemne for Britt Viola Danielsen Testemne ...
4,rher@hvl.no,4886,Rune Herheim,Testemne for Rune Herheim Testemne for Beate...


In [14]:
data['lag_nytt'] = data.apply(lambda row: str(row['namn']) in str(row['testemne']), axis=1)

In [16]:
data['lag_nytt'].value_counts()

lag_nytt
True     1416
False      53
Name: count, dtype: int64

Den neste kodesnutten går gjennom alle eg har funnet og oppretter nye testemne. Ikkje køyr denne fleire ganger!

In [ ]:
# for n in range(11,len(lag_testemne)):
#     i = lag_testemne.iloc[n]
#     params = {
#         'course[name]': f"Testemne for {i['Namn']}",
#         'course[sis_course_id]': f"hvl250506-e{n}-5-akv",
#         'course[course_code]': f"Testemne for {i['Namn']}",
#         'course[syllabus_body]': "Dette er ditt testemne; her kan du gjere alt du vil prøve på i eit 'vanleg' emne.\nTing du lager her kan du hente inn i andre emne til bruk i undervising.",
#         'course[default_view]': 'syllabus', 
#     }
#     url = f"https://hvl.instructure.com/api/v1/accounts/9044/courses"
#     response = requests.post(url, headers=headers, params=params)
#     course_id = response.json()['id']
#     params = {
#         'enrollment[user_id]': i['user_id'],
#         'enrollment[type]': 'TeacherEnrollment',
#         'enrollment[workflow_state]': 'active',
#     }
#     url = f"https://hvl.instructure.com/api/v1/courses/{course_id}/enrollments"
#     response = requests.post(url, headers=headers, params=params)
#     print(f"{i['user_id']}: {response.status_code}")

# Neste steg: sjekke nye lærarar kvar natt

No må eg lage meg ei kontrolliste med kven som har testemne. Kvar natt sjekker eg så importen av lærarar, og ser om det er nokon der som ikkje er på lista over eksisterande lærarar. Desse skal i så fall få eit testemne etter malen over.

Først må eg lese fila over alle som vert overført i natt frå FS:

In [ ]:
url = "https://hvl.instructure.com/api/v1/accounts/1/sis_imports"
svar = requests.get(url, headers=headers, params=params)
importerte_filar = svar.json()

no = datetime.now()
igår = (datetime.now() - timedelta(hours=24)).isoformat()
filliste = []
for i in importerte_filar['sis_imports']:
    import_dato = datetime.fromisoformat(i['created_at'].replace('Z', '+00:00')).replace(tzinfo=None)
    igår_dato = datetime.fromisoformat(igår)
    if (import_dato > igår_dato):
        for j in i['csv_attachments']:
            if j['filename'] == "users_filtered.csv":
                filliste.append({'dato': i['created_at'], 'url': j['url']})
users_liste = []
for i in filliste:
    temp = pd.read_csv(StringIO(requests.get(i['url'], headers=headers).content.decode("utf-8")))
    print(f"{i['dato']}: {len(temp)}")
    users_liste.append(temp)

overført = pd.concat(users_liste)
overført['brukarnamn'] = overført['login_id'].apply(lambda x: x.split('@')[0])
overført = overført[overført['brukarnamn'].str.isalpha()]
nye = overført[~overført['brukarnamn'].isin(eksisterande['brukarnamn'])]

In [ ]:
nye

Dersom eg ser at det er nye tilsette skal eg lage nye testemner for dei, og eg må legge dei til i fila `users_filtered.csv`: 

In [ ]:
dato = datetime.now().strftime("%y%m%d")
data = pd.read_csv("users_filtered.csv")
n = 1
for row, index in nye.iterrows():
#     params = {
#         'course[name]': f"Testemne for {row['Namn']}",
#         'course[sis_course_id]': f"hvl{dato}-e{n}-akv",
#         'course[course_code]': f"Testemne for {row['Namn']}",
#         'course[syllabus_body]': "Dette er ditt testemne; her kan du gjere alt du vil prøve på i eit 'vanleg' emne.\nTing du lager her kan du hente inn i andre emne til bruk i undervising.",
#         'course[default_view]': 'syllabus', 
#     }
#     url = f"https://hvl.instructure.com/api/v1/accounts/9044/courses"
#     response = requests.post(url, headers=headers, params=params)
#     course_id = response.json()['id']
#     params = {
#         'enrollment[user_id]': i['user_id'],
#         'enrollment[type]': 'TeacherEnrollment',
#         'enrollment[workflow_state]': 'active',
#     }
#     n += 1
#     url = f"https://hvl.instructure.com/api/v1/courses/{course_id}/enrollments"
#     response = requests.post(url, headers=headers, params=params)
#     print(f"{i['user_id']}: {response.status_code}")

    # Legg nye til i `users_filtered.csv`
    data.loc[len(data)] = [index[0], index[1], index[2], index[3], index[4], index[5], index[6], index[7]]

data.to_csv("users_filtered.csv", index=False)